In [194]:
import requests
from bs4 import BeautifulSoup,Comment
import pandas as pd
import csv

In [4]:
# Convert the csv file into dataframe for more convenience

dataframe = pd.read_csv('destinations_LP_crawler_Ex5.csv')

         city                           city_LP
0     Antalya       mediterranean-coast/antalya
1      Athens                            Athens
2   Ayia-napa  the-republic-of-cyprus/Agia-napa
3     Bangkok                           Bangkok
4   Barcelona                         Barcelona
..        ...                               ...
71     Venice                            Venice
72     Vienna                            Vienna
73     Warsaw                            Warsaw
74  Zakynthos                         Zakynthos
75     Zurich                            Zurich

[76 rows x 2 columns]


In [301]:
# Creates an empty output dataframe

column_names = ['city','country','description']
LP_destinations = pd.DataFrame(columns = column_names)

In [302]:
# API key for google geocode service

YOUR_API_KEY = '12345'

# Itarete
for index, row in dataframe.iterrows():
    city = row['city']
    city_LP = row['city_LP']
    
    geocode_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={city}&key={YOUR_API_KEY}'
    response = requests.get(geocode_url).json()
    if response['status'] != 'OK':
        geocode_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={city_LP}&key={YOUR_API_KEY}'
        response = requests.get(geocode_url).json()
    country = response['results'][0]['address_components'][-1]['long_name']
    
    if (country >= '0' and country <= '9'):
        country = response['results'][0]['address_components'][-2]['long_name']
    if country == 'United States':
        country = 'usa'
    if city == 'Krak�':
        country = 'poland'
    
    # Scraping from the certain url
    LP_url = f'https://www.lonelyplanet.com/{country}/{city_LP}'
    
    # 
    webpage_response = requests.get(LP_url)
    webpage = webpage_response.content
    soup = BeautifulSoup(webpage,"html.parser")
    
    # This div contains the description
    div_content = soup.find('div',{'class':'readMore_content___5EuR relative overflow-hidden max-h-96 is-max wysiwyg'})
    
    # Iterate thorugh different paragraphs and save it into new variable called "description"
    description = ""
    
    for child in div_content.children:
        if child != '\n':
            if child.name == 'h2':
                description += child.string + ": "
            else:
                if child.string==None:
                    text = " ".join(child.find_all(text=lambda t: not isinstance(t, Comment)))
                    description += text + "\n"
                else:
                    description += child.string + "\n"
                
    # Appending new information at the end of the output dataframe
    new_row = [city, country, description]
    LP_destinations.loc[len(LP_destinations.index)] = new_row

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


In [311]:
# Export output dataframe into a csv file

LP_destinations.to_csv('LP_destinations.csv', index=False)